## Setting Libraries

In [ ]:
!python --version

Python 3.7.12


In [ ]:
!pip install pydub
!pip install pytube
#!pip install subprocess

     |████████████████████████████████| 56 kB 2.9 MB/s 


In [ ]:
import os
import json
import subprocess
from pytube import YouTube # 유튜브 크롤링 모듈
from pytube.exceptions import VideoUnavailable #유효하지 않을떄
from pydub import AudioSegment # 오디오파일 받아오는 모듈
from pydub.utils import make_chunks
from IPython import display as ipd

In [ ]:
from google.colab import drive
drive.mount('/content/MyDrive')

Mounted at /content/MyDrive


In [ ]:
BASE_DIR = '/content/MyDrive/MyDrive/Colab Notebooks/[해커톤]DeepASMR/'
MP4_SOURCE = BASE_DIR + '[code]yt_output'
WAV_SOURCE = BASE_DIR + '[code]yt_wav'

class YoutubeDownloader():
    def __init__(self, folder):
        self.folder = folder

    def down_files(self, urls):
        if type(urls) == list and len(urls):
            for idx, url in enumerate(urls):
                try:
                    yt = YouTube(url) #yt 인스턴스 생성
                except VideoUnavailable:
                    print(f'{idx + 1}번째 {url}는 사용할 수 없습니다.') # url이 사용불가면 에러나니까
                else:
                    audio = yt.streams.filter(only_audio=True).first() #audio에 음원파일이 들어감
                    print(f'파일명: {yt.title}\n') 
                    print(f'다운로드 시작')
                    try:
                        filename = input("원하는 파일명.mp4를 입력하세요. ")
                        audio.download(output_path=self.folder, filename=filename)
                        print(f'다운로드 완료 {(idx + 1)} / {len(urls)}')
                        print("-" * 100)
                    except Exception:
                        print(f"다운로드 중 오류가 발생하였습니다.")
                        continue
            print("파일 다운로드 종료")
            return
        
        else:
            print('URL을 올바르게 입력해주세요')
            return

In [ ]:
# 가져올 Youtube 영상 url을 urls 배열에 저장  
urls = ['https://youtu.be/rMQTPyo_RbQ']
len(urls)

1

In [ ]:
# YoutubeDownloader 객체 생성
yt = YoutubeDownloader(MP4_SOURCE)

In [ ]:
# urls 배열을 받아 다운로드 진행
yt.down_files(urls)

파일명: ASMR A Standard of Ear Cleaning 귀청소의 정석

다운로드 시작
원하는 파일명.mp4를 입력하세요. asmr_earclean_1.mp4
다운로드 완료 1 / 1
----------------------------------------------------------------------------------------------------
파일 다운로드 종료


In [ ]:
class AudioHandler():
    def __init__(self, filepath):
        self.file_path = filepath # 오디오 파일 전체 경로
        self.file_name = os.path.splitext(self.file_path)[0] # 파일경로
        self.file_format = os.path.splitext(self.file_path)[1] # 확장자
        self.__audio = AudioSegment.from_file(self.file_path) # 오디오 파일 로드
    
    def get_duration(self):
        """ 원본 오디오 파일의 재생시간"""
        return self.__audio.duration_seconds
    
    def file_split(self, start_min, start_sec, end_min, end_sec, fname):
        """ 원본 오디오 파일에서 원하는 구간 Split 해서 wav 파일로 저장"""
        t1 = start_min * 60 * 1000 + start_sec * 1000
        t2 = end_min * 60 * 1000 + end_sec * 1000
        split_audio = self.__audio[t1:t2]
        save_path = os.path.join(WAV_SOURCE, fname)
        split_audio.export(save_path + '.wav', format='wav')
            
    def convert_to_wav(self):
        """mp4 파일을 wav로 포맷변경"""
        try:
            filename = os.path.basename(self.file_name)
            save_path = os.path.join(WAV_SOURCE, filename)
            self.__audio.export(save_path + ".wav", format="wav")
        except:
            print("파일변환에러")

    def chunk_wav_data(self, chunk_length):
        """chunk_length 단위로 원본 오디오 파일을 temp 폴더에 저장"""
        folder_path = os.path.join(BASE_DIR, 'temp')
        if os.path.exists(BASE_DIR + '/temp'): # temp 폴더가 이미 존재한다면 폴더 삭제
            os.system("rm -rf {}".format(folder_path))

        os.mkdir(folder_path) # temp 폴더 생성
        chunk_length_ms = 1000 * chunk_length # pydub calculates in millisec
        chunks = make_chunks(self.__audio, chunk_length_ms) #Make chunks of one sec      

        for i, chunk in enumerate(chunks):
            chunk_name = "{}.wav".format(i)
            print("exporting ", chunk_name)
            chunk.export(os.path.join(folder_path, chunk_name), format="wav")

In [ ]:
# AudioHandler 객체 생성
audio = AudioHandler(MP4_SOURCE + '/asmr_earclean_1.mp4')

In [ ]:
audio.file_path

'/content/MyDrive/MyDrive/Colab Notebooks/[해커톤]DeepASMR/[code]yt_output/asmr_earclean_1.mp4'

In [ ]:
audio.file_name

'/content/MyDrive/MyDrive/Colab Notebooks/[해커톤]DeepASMR/[code]yt_output/asmr_earclean_1'

In [ ]:
audio.file_format

'.mp4'

In [ ]:
# MP4 -> WAV 변환
audio.convert_to_wav()
os.listdir(WAV_SOURCE)

['asmr_mix_20210608.wav',
 'asmr_talk_whispering_191004.wav',
 'asmr_talk_wordrepeat_200721.wav',
 '.ipynb_checkpoints',
 'asmr_soft_mouthandbrush_211012.wav',
 'asmr_soft_tongue_210908.wav',
 'asmr_talk_hairsalon_170926.wav',
 'asmr_tap_earcleaning_210617.wav',
 'asmr_tap_soap_210327.wav',
 'asmr_tap_wood_210930.wav',
 'asmr_1.wav',
 'asmr_nature_fire_20211014.wav',
 'asmr_earclean_1.wav']

In [ ]:
# 원본 파일에서 0분 30초 ~ 1분 30초 구간을 현재 폴더에 'asmr_1.wav' 라는 파일명으로 저장
audio.file_split(0, 30, 1, 30, 'asmr_earclean_split')
os.listdir(WAV_SOURCE)

['asmr_mix_20210608.wav',
 'asmr_talk_whispering_191004.wav',
 'asmr_talk_wordrepeat_200721.wav',
 '.ipynb_checkpoints',
 'asmr_soft_mouthandbrush_211012.wav',
 'asmr_soft_tongue_210908.wav',
 'asmr_talk_hairsalon_170926.wav',
 'asmr_tap_earcleaning_210617.wav',
 'asmr_tap_soap_210327.wav',
 'asmr_tap_wood_210930.wav',
 'asmr_1.wav',
 'asmr_nature_fire_20211014.wav',
 'asmr_earclean_1.wav',
 'asmr_earclean_split.wav']

In [ ]:
# 30 초 단위로 audio chunk
audio.chunk_wav_data(6)

FileExistsError: ignored